# Northern Patagonia Load
## Organize info
### Paul Sandoval Quilodrán 
#### https://github.com/SQPaul/NPLoad - paul.sandoval97@gmail.com

### import packages

In [1]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

import warnings
warnings.simplefilter("ignore", category=FutureWarning)

# 1.- Streamflow series (data extracted in situ)
## info from Brian

In [9]:
#Read sheet_names
path = '/media/phi/Hard_disk2/Projects/Pisciculture_vs_Rivers/Entrega/raw_data/FSEQ normas/2. Hidro 2022 FSEQ_EDITED.xlsx'
sheet_names = pd.ExcelFile(path).sheet_names
sheet_names = sheet_names[0:5]

names = ["Puelo","Yelcho","Palena","Cisnes","Aysen"]
q = []

#Iterate in sheets
for i in sheet_names: 
    #Read excel
    xlsx = pd.read_excel(path,sheet_name=str(i),index_col="date")
    #Delete spaces in dates
    xlsx.index = xlsx.index.str.strip()
    #Date to correct date format
    xlsx.index = pd.to_datetime(xlsx.index,format='%d/%m/%Y %H:%M')
    #Calculate daily streamflow
    xlsx = xlsx.resample("D").mean()
    #Reindex to assure a complete time series 
    date_range = pd.date_range(start=pd.to_datetime("2022-01-01"),end=pd.to_datetime("2022-12-31"))
    xlsx = xlsx.reindex(date_range)
    #Append data
    q.append(xlsx.iloc[:,1])
    
    #Plot streamflow ts
    fig = px.line(xlsx, x=xlsx.index, y=xlsx.iloc[:,1],title=str(i))
    fig.update_layout(xaxis_title="",yaxis_title="m3/s")
    fig.show()
    
#Create final DF     
for i in range(len(names)):
    if i == 0:
        df = pd.DataFrame(q[i],q[i].index)
        df.columns = [str(names[i])]
    else:
        df[str(names[i])] = q[i]

df.index.name = "date"
#df
#df.to_csv('/media/phi/Hard_disk2/Projects/Pisciculture_vs_Rivers/Entrega/Timeseries/q.csv')

# 2.- Quality time series
## info from Brian

In [3]:
#Read input data
q_path = '/media/phi/Hard_disk2/Projects/Pisciculture_vs_Rivers/Entrega/Timeseries/q.csv'
quality_path = '/media/phi/Hard_disk2/Projects/Pisciculture_vs_Rivers/Entrega/raw_data/FSEQ normas/info_brian.xlsx'

##quality
quality = pd.read_excel(quality_path,sheet_name="Calidad")
quality = quality.groupby(['River','Param'])

##streamflow
q = pd.read_csv(q_path,index_col="date")
###groupby function sort the data in alphabetic order, so is necessary sort the q DF
q = q.sort_index(axis=1)
###create name list
names = q.columns.to_list()

#From quality DF create parameter and values list
##Create empty list
param_names = []
param_values = []

for i in quality:
    var_name = str(i[0][0])+"_"+str(i[0][1])
    param_names.append(var_name)
    param_values.append(pd.DataFrame(i[1]))

names_all = []
dn_all = []
don_all = []
dop_all = []
nh4_all = []
nox_all = []
pO4_all = []

##Adjusment value to create de DFs
adj_value = 6

#Create a list per parameter
for i in range(len(names)):    
    #names
    names_all.append(names[i])#[:-4])
    #dn
    dn = pd.DataFrame(param_values[adj_value*i].iloc[:,1])
    dn.index = param_values[adj_value*i].iloc[:,0] 
    dn_all.append(dn)
    #don
    don = pd.DataFrame(param_values[adj_value*i+1].iloc[:,1])
    don.index = param_values[adj_value*i+1].iloc[:,0] 
    don_all.append(don)
    #dop
    dop = pd.DataFrame(param_values[adj_value*i+2].iloc[:,1])
    dop.index = param_values[adj_value*i+2].iloc[:,0] 
    dop_all.append(dop)
    #nh4
    nh4 = pd.DataFrame(param_values[adj_value*i+3].iloc[:,1])
    nh4.index = param_values[adj_value*i+3].iloc[:,0] 
    nh4_all.append(nh4)
    #nox
    nox = pd.DataFrame(param_values[adj_value*i+4].iloc[:,1])
    nox.index = pd.to_datetime(param_values[adj_value*i+4].iloc[:,0]) 
    nox_all.append(nox)
    #p04
    pO4 = pd.DataFrame(param_values[adj_value*i+5].iloc[:,1])
    pO4.index = pd.to_datetime(param_values[adj_value*i+5].iloc[:,0]) 
    pO4_all.append(pO4)

#Create a list per station
df_names = ["q","DN","DON","DOP","NH4","N0x","P-PO4"]

for i in range(len(names)):
    q_act = q.iloc[:,i]
    q_act.index = pd.to_datetime(q_act.index)
    dn_act = dn_all[i]
    dn_act.index = pd.to_datetime(dn_act.index,format='%d/%m/%Y')
    don_act = don_all[i]
    don_act.index = pd.to_datetime(don_act.index,format='%d/%m/%Y')
    dop_act = dop_all[i]
    dop_act.index = pd.to_datetime(dop_act.index,format='%d/%m/%Y')
    nh4_act = nh4_all[i]
    nh4_act.index = pd.to_datetime(nh4_act.index,format='%d/%m/%Y')
    nox_act = nox_all[i]
    nox_act.index = pd.to_datetime(nox_act.index,format='%d/%m/%Y')
    pO4_act = pO4_all[i]
    pO4_act.index = pd.to_datetime(pO4_act.index,format='%d/%m/%Y')
    df = pd.concat([q_act,dn_act,don_act,dop_act,nh4_act,nox_act,pO4_act],axis=1,join="outer")
    df.columns = df_names
    df.index.name = "date"
    csv_name = '/media/phi/Hard_disk2/Projects/Pisciculture_vs_Rivers/Entrega/Timeseries//'+str(q_act.name)+"_q-vs-quality.csv"
    df.to_csv(csv_name)
    print(csv_name)

/media/phi/Hard_disk2/Projects/Pisciculture_vs_Rivers/Entrega/Timeseries//Aysen_q-vs-quality.csv
/media/phi/Hard_disk2/Projects/Pisciculture_vs_Rivers/Entrega/Timeseries//Cisnes_q-vs-quality.csv
/media/phi/Hard_disk2/Projects/Pisciculture_vs_Rivers/Entrega/Timeseries//Palena_q-vs-quality.csv
/media/phi/Hard_disk2/Projects/Pisciculture_vs_Rivers/Entrega/Timeseries//Puelo_q-vs-quality.csv
/media/phi/Hard_disk2/Projects/Pisciculture_vs_Rivers/Entrega/Timeseries//Yelcho_q-vs-quality.csv
